![](image_text/110a_02.png)

![](image_text/110a_03.png)

![](image_text/110a_04.png)

![](image_text/110a_05.png)

![](image_text/110a_06.png)

![](image_text/110a_07.png)

![](image_text/110a_08.png)

![](image_text/110a_09.png)

![](image_text/110a_10.png)

![](image_text/110a_11.png)

以下で、左に表示される入出力セル番号は新規実行した場合の番号を示します。
説明とセル番号を合わせるには、例えば、上のメニューの[Kernel]->[Restart Kernel and Run all Cells...]を実行をしてください。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
DR_TYPE = "pca" # "pca", "smearing", "none"
NDIM = 6 # 次元圧縮後の説明変数次元（数）
RANDOM_STATE = 5 # K-means法で用いる乱数SEED
SECOND_DR_TYPE = "none" # "tsne", "none", or "pca"

In [ ]:
ROOT = ".."
filename = f"{ROOT}/data/hea4_dos.csv"
df = pd.read_csv(filename, index_col=None) #観測データ読み込み
# "semicore"カラムにセミコア元素が含まれる
TARGET_NAME = 'semicore' # 目的変数では無いが答え合わせに用いるカラム名
semicore_list = np.unique(df[TARGET_NAME].values)
# np.uniqueで重複が無い全セミコア元素名を取得する。
print("semicore_lit", semicore_list)
n_clusters = semicore_list.size # 全セミコア元素数=12を取得
print("n_clusters", n_clusters)
logdos_names = [] # 説明変数カラムに用いる変数定義
for i in range(100):
    logdos_names.append("log10_dos{}".format(i+1))# 説明変数カラム名を生成する。

In [ ]:
from dos_clustering_misc import plot_each_DOS 
# ファイル"./dos_clustering_misc.py"にあるユーザー定義のplot_each_DOS関数をimportする。
plot_each_DOS(df, logdos_names, TARGET_NAME) # セミコア元素毎にDOSスペクトルデータを可視化する。
# Bi-Zrまでそれぞれ５０データインスタンスのDOSを示し、特徴あるpeak,dip,全体の形を持つ。
# 特に、HgとMoはpeakはよく見えないが、全体の形が異なる特徴を持つ。（高エネルギー側にばらけたピークを持つ。）

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from dos_clustering_misc import add_convolution_variables
if DR_TYPE == "pca":
    _X = df[logdos_names].values #_X(N,P)、生説明変数
    dr = PCA(NDIM)  # カラム毎規格化を行わずにPCAで次元圧縮、圧縮後次元がNDIM
    X_dr = dr.fit_transform(_X) # X_dr(N,NDIM)
elif DR_TYPE == "smearing":
    df, smearedlogdos_names = add_convolution_variables(
        df, logdos_names, NDIM)
    X_dr = df[smearedlogdos_names].values
else: # pcaでもsmearingでもない場合,noneを含む、全次元を用いる。
    X_dr = df[logdos_names].values # _X(N,P)
    NDIM = len(logdos_names) # NDIM=P
y = df[TARGET_NAME].values # 答え合わせのためのラベル

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_dr = scaler.fit_transform(X_dr) #　X_drを次元圧縮後に更に規格化

In [ ]:
from sklearn.cluster import KMeans
if SECOND_DR_TYPE in ["tsne", "pca"]:
    print("second_dr_type", SECOND_DR_TYPE)
    if SECOND_DR_TYPE == "tsne":
        dr2 = TSNE(2,  init="pca", random_state=RANDOM_STATE)
    elif SECOND_DR_TYPE == "pca":
        dr2 = PCA(2, random_state=RANDOM_STATE)
    X2 = dr2.fit_transform(X_dr)
    km = KMeans(n_clusters, random_state=RANDOM_STATE)
    yp = km.fit_predict(X2)
else:
    print("second_dr_type_none")
    km = KMeans(n_clusters, random_state=RANDOM_STATE) 
    # K-means法をクラスター数n_clusterで行う。
    yp = km.fit_predict(X_dr) # yp(N)、クラスターIDが入る
    dr2 = TSNE(2,  init="pca", random_state=RANDOM_STATE) 
    # 可視化用にt-sneで再度次元圧縮する。
    X2 = dr2.fit_transform(X_dr) # X2(N,2), 可視化用の説明変数空間

In [ ]:
from dos_clustering_misc import plot_X2_compare, assign_frequent_value
yp_freq = assign_frequent_value(y, yp) # yは文字列、ypは数字なので、最頻度でそれらを当てはめる。
# yp_freq(N):文字列、yに対応した文字列
plot_X2_compare(X2, y, yp_freq) 
# 横軸が次元圧縮した一次元目と二次元目
# 左パネルが(X2, y)を表す。yは色と文字列で表す。
# 右パネルが(X2, y_freq)を表す。y_freqは色と文字列で表す。

In [ ]:
from sklearn.metrics import confusion_matrix
y_uniq = np.unique(y) # confusion_matrix関数内部と同じlabelの並び替えを行う。（version upで変わる可能性あり。）
cm = confusion_matrix(y, yp_freq, labels=y_uniq) # 形式的に混同行列を求める。
pd.DataFrame(cm, index=y_uniq, columns=y_uniq) #データフレーム形式にして表示する。

In [ ]:
from sklearn.metrics import classification_report
msg = classification_report(y,yp_freq) # 各種分類評価指標を形式的に計算する。
print(msg)
# yとypで対応した名前がない場合は以下でzero_division警告が表示されますが問題ありません。

![](image_text/110a_12.png)

![](image_text/110a_13.png)

![](image_text/110a_14.png)

![](image_text/110a_15.png)

![](image_text/110a_16.png)

![](image_text/110a_17.png)